<a href="https://colab.research.google.com/github/yashx07/RagReading1Pdf/blob/main/RagFinance1pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Alternative virtualenv method
!pip install virtualenv
!virtualenv --python=python3 /content/rag_env
!source /content/rag_env/bin/activate


created virtual environment CPython3.11.11.final.0-64 in 255ms
  creator CPython3Posix(dest=/content/rag_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.3.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
!pip install -q pymupdf groq semantic-router pinecone-client datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [4]:
import os
import getpass
import time
from pathlib import Path
import fitz  # PyMuPDF
from tqdm.auto import tqdm
from datasets import Dataset
from semantic_router.encoders import HuggingFaceEncoder
from pinecone import Pinecone, ServerlessSpec
from groq import Groq

In [5]:
# %% PDF Processing Function
def process_pdf(pdf_path):
    """Process PDF into text chunks with metadata"""
    doc = fitz.open(pdf_path)
    chunks = []

    for page_num, page in enumerate(doc):
        text = page.get_text()
        chunks.append({
            "id": f"page_{page_num+1}",
            "text": text,
            "metadata": {
                "title": "Union Budget 25-26",
                "content": text,
                "page": page_num+1
            }
        })
    return chunks


In [6]:
# %% Process your PDF document
pdf_path = "/content/Union Budget 25-26 - HSIE-202502012246365785408.pdf"
data = process_pdf(pdf_path)

In [7]:
# %% Initialize encoder (run on CPU)
encoder = HuggingFaceEncoder(name="dwzhu/e5-base-4k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/225M [00:00<?, ?B/s]

In [8]:
# %% Pinecone Setup
api_key = os.getenv("PINECONE_API_KEY") or getpass.getpass("Pinecone API Key: ")
pc = Pinecone(api_key=api_key)

index_name = "union-budget-rag"
spec = ServerlessSpec(cloud="aws", region="us-east-1")

Pinecone API Key: ··········


In [52]:
# Correct usage for semantic-router's HuggingFaceEncoder
from semantic_router.encoders import HuggingFaceEncoder

# Initialize with your chosen model
encoder = HuggingFaceEncoder(name="krlng/sts-GBERT-bi-encoder")

# Get embedding dimension (no .encode() needed)
sample_text = ["This is a sample text."]  # Must be list of strings
embedding = encoder(sample_text)  # Direct call instead of .encode()
dimension = len(embedding[0])

dimension



In [10]:
# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=spec
    )

index = pc.Index(index_name)

In [16]:
# %% Upsert PDF vectors to Pinecone
batch_size = 16

for i in tqdm(range(0, len(data), batch_size)):
    batch = data[i:i+batch_size]
    vectors = []

    for item in batch:
        vectors.append({
            "id": item["id"],
            "values": encoder([item["text"]])[0],
            "metadata": item["metadata"]
        })

    index.upsert(vectors=vectors)


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
# %% Initialize Groq Client
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or getpass.getpass("Groq API Key: ")
groq_client = Groq()

In [18]:
# %% RAG Query Functions
def retrieve_docs(query, top_k=3):
    """Retrieve relevant document chunks"""
    query_embedding = encoder([query])[0]
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return [match.metadata["content"] for match in results.matches]

def generate_answer(query):
    """Generate answer using Llama 3-70B"""
    context = retrieve_docs(query)
    system_prompt = f"""You are a budget analysis expert. Use this context:
    {''.join(context)}

    Answer in bullet points with page references. Be extremely precise."""

    response = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content


In [49]:

# %% Example Query Execution
query = "what do you know about consumer in contributing analyst?"
print(generate_answer(query))


Based on the provided document, here's what I know about the "Consumer" sector in the Contributing Analyst section:

* There are two analysts covering the Consumer sector:
	+ Vishal Gutka (email: vishal.gutka@hdfcsec.com, phone: +91-22-6171-7324)
	+ Anuj D (email: anuj.d@hdfcsec.com, phone: +91-22-6171-7357)

(Page reference: Page 9)


In [50]:

# %% Example Query Execution
query = "what are the budget highlights for oil and gas?"
print(generate_answer(query))


Here are the budget highlights for Oil and Gas:

• FY26BE total subsidy provision is INR 106bn for LPG compared to FY25RE total LPG subsidy of INR 132bn. (Page 23)
• LPG DBT provision is INR 15bn compared to FY25RE LPG DBT provision of INR 5bn. (Page 23)
• Considering the current under-recovery on LPG, the subsidy provision is inadequate. (Page 23)
• Negative for Oil Marketing Companies. (Page 23)
